<a href="https://colab.research.google.com/github/ayushabhinav/TechGig/blob/main/Telstra_Network_Disruptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Problem Statement
- please refer the link
  - https://www.techgig.com/practice/question/log-loss-multi-class-log-loss/TWZQckxGcjAwUXl3L2dOVUZLenRqWWhnSjU2NnF3NkNkWml3eWJzTTdtWVIzZ2NGS2wrNDRxQUpSejZiL29NTQ==/1

#### Connect to Drive to load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Import Libraries

In [ ]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Load data into DataFrame

In [ ]:
os.listdir('/content/drive/MyDrive/TechGeek_DS_problem_Telstra_Network_Disruptions')

['train.csv',
 'test.csv',
 'event-type.csv',
 'log-feature.csv',
 'resource-type.csv',
 'severity-type.csv',
 'sample-submission.csv',
 'Telstra_Network_Disruptions_G4GProb.ipynb',
 'Untitled0.ipynb']

In [ ]:
base_folder = ('/content/drive/MyDrive/TechGeek_DS_problem_Telstra_Network_Disruptions')
train_file = os.path.join(base_folder, 'train.csv')
test_file = os.path.join(base_folder, 'test.csv')
event_file = os.path.join(base_folder, 'event-type.csv')
log_feature_file = os.path.join(base_folder, 'log-feature.csv')
resource_type_file = os.path.join(base_folder, 'resource-type.csv')
severity_type_file = os.path.join(base_folder, 'severity-type.csv')

### Analyse each file and do feature engineering

#### Train File
- Contains Id and location. Id is primary key and foreign key to other file.
- No Repeatation of ID

In [ ]:
train_data = pd.read_csv(train_file)
train_data.head()

,id,location,fault_severity
0,14121,location 118,1
1,9320,location 91,0
2,14394,location 152,1
3,8218,location 931,1
4,14804,location 120,0


In [ ]:
train_data.shape

(5999, 3)

In [ ]:
# check the unique ids
train_data.id.value_counts().sort_values(ascending=False)

14121    1
7965     1
8218     1
14804    1
1080     1
        ..
4587     1
6846     1
14207    1
6239     1
15709    1
Name: id, Length: 5999, dtype: int64

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5999 entries, 0 to 5998
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5999 non-null   int64 
 1   location        5999 non-null   object
 2   fault_severity  5999 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 140.7+ KB


In [ ]:
train_data.location.value_counts().sort_values(ascending=False)

location 821     75
location 1107    61
location 734     58
location 1008    57
location 810     56
                 ..
location 654      1
location 1125     1
location 158      1
location 525      1
location 678      1
Name: location, Length: 887, dtype: int64

In [ ]:
filt = train_data.location == 'location 821'
train_data[filt]

,id,location,fault_severity
156,15486,location 821,0
170,8611,location 821,2
175,6225,location 821,1
234,7295,location 821,1
280,2380,location 821,1
...,...,...,...
5490,486,location 821,0
5559,8617,location 821,0
5568,818,location 821,0
5899,9076,location 821,2


In [ ]:
from sklearn.preprocessing import LabelEncoder

location_encoder = LabelEncoder()
train_data['location_enc'] = location_encoder.fit_transform(train_data.location)
train_data

,id,location,fault_severity,location_enc
0,14121,location 118,1,129
1,9320,location 91,0,812
2,14394,location 152,1,161
3,8218,location 931,1,830
4,14804,location 120,0,132
...,...,...,...,...
5994,15060,location 803,1,712
5995,4587,location 879,0,781
5996,6846,location 976,1,869
5997,14207,location 1053,1,51


#### Event File
- Contains ID ans event_type
- Repeation of ID indicates multile event type are assiciated with one ID. 
- Approach
  - Binarize the event_type 
  - group by ID  and sum operation
- Result
  - Remove duplication of ID. one row per id 
  - rows are sparse with 1 at event position associated with ID

In [ ]:
event_data = pd.read_csv(event_file)
event_data.head()

,id,event_type
0,6597,event_type 11
1,8011,event_type 15
2,2597,event_type 15
3,5022,event_type 15
4,5022,event_type 11


In [ ]:
event_data.shape

(31170, 2)

In [ ]:
event_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31170 entries, 0 to 31169
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          31170 non-null  int64 
 1   event_type  31170 non-null  object
dtypes: int64(1), object(1)
memory usage: 487.2+ KB


In [ ]:
event_data.id.value_counts().sort_values(ascending=False)

10354    11
15184    10
14243    10
215       9
8590      9
         ..
17826     1
8780      1
70        1
7807      1
4464      1
Name: id, Length: 18552, dtype: int64

In [ ]:
event_data_enc = pd.get_dummies(event_data).groupby(by='id').sum()
event_data_enc

,event_type_event_type 1,event_type_event_type 10,event_type_event_type 11,event_type_event_type 12,event_type_event_type 13,event_type_event_type 14,event_type_event_type 15,event_type_event_type 17,event_type_event_type 18,event_type_event_type 19,...,event_type_event_type 5,event_type_event_type 50,event_type_event_type 51,event_type_event_type 52,event_type_event_type 53,event_type_event_type 54,event_type_event_type 6,event_type_event_type 7,event_type_event_type 8,event_type_event_type 9
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18548,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18550,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### log_feature_file
- Contains ID, features and Volume
- Repeation of Id indicates, muliple features are associated with oen ID and volumn indicates particular feature is associated many times.
- Approach
  - LabelBinarize the features
  - multiply is by volumn 
  - group by ID and sum it
- Result
  - Remove duplication of ID. one row per id
  - rows are sparse with volumn count at feature position associated with ID

In [ ]:
log_data = pd.read_csv(log_feature_file)
log_data

,id,log_feature,volume
0,6597,feature 68,6
1,8011,feature 68,7
2,2597,feature 68,1
3,5022,feature 172,2
4,5022,feature 56,1
...,...,...,...
58666,8720,feature 209,1
58667,6488,feature 54,3
58668,878,feature 62,1
58669,4464,feature 209,1


In [ ]:
log_data.shape

(58671, 3)

In [ ]:
log_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58671 entries, 0 to 58670
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           58671 non-null  int64 
 1   log_feature  58671 non-null  object
 2   volume       58671 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [ ]:
log_data.id.value_counts().sort_values(ascending=False)

5233     20
15184    20
12575    19
9266     18
4593     18
         ..
6308      1
10270     1
16217     1
5317      1
6597      1
Name: id, Length: 18552, dtype: int64

In [ ]:
filt = log_data.id == 5233     
log_data[filt].sort_values(by='log_feature')

,id,log_feature,volume
26386,5233,feature 227,3
26380,5233,feature 228,35
26372,5233,feature 229,28
26374,5233,feature 230,20
26384,5233,feature 231,1
26377,5233,feature 232,68
26387,5233,feature 233,15
26373,5233,feature 234,28
26383,5233,feature 235,12
26370,5233,feature 236,3


In [ ]:
from sklearn.preprocessing import  LabelBinarizer

feature_binarizer  = LabelBinarizer()

log_feature_binary = feature_binarizer.fit_transform(log_data.log_feature)
log_feature_binary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
t = log_feature_binary * np.array(log_data.volume.values).reshape(-1, 1)
t.shape

(58671, 386)

In [ ]:
log_data.id.values.reshape(-1, 1).shape

(58671, 1)

In [ ]:
log_data_enc= pd.DataFrame(np.hstack([log_data.id.values.reshape(-1, 1), t]))
log_data_enc.rename(columns={0:'id'}, inplace=True)
log_data_enc.set_index('id', inplace=True)

In [ ]:
np.argmax(log_data_enc.values, axis=1).reshape(-1,1)[:10]

array([[351],
       [351],
       [351],
       [ 81],
       [338],
       [104],
       [355],
       [114],
       [338],
       [365]])

In [ ]:
log_data.head(10)

,id,log_feature,volume
0,6597,feature 68,6
1,8011,feature 68,7
2,2597,feature 68,1
3,5022,feature 172,2
4,5022,feature 56,1
5,5022,feature 193,4
6,5022,feature 71,3
7,6852,feature 201,2
8,6852,feature 56,1
9,6852,feature 80,2


In [ ]:
log_data_enc.index.value_counts().sort_values(ascending=False)

5233     20
15184    20
12575    19
9266     18
4593     18
         ..
6308      1
10270     1
16217     1
5317      1
6597      1
Name: id, Length: 18552, dtype: int64

In [ ]:
log_data_enc_grp = log_data_enc.groupby(by='id').sum()
log_data_enc_grp

,1,2,3,4,5,6,7,8,9,10,...,377,378,379,380,381,382,383,384,385,386
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18548,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
filt = log_data_enc_grp.index == 5233     
log_data_enc_grp[filt].values

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3, 35,
        28,  0, 20,  1, 68, 15, 28, 12,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [ ]:
filt = log_data.id == 5233     
log_data[filt].sort_values(by='log_feature')

,id,log_feature,volume
26386,5233,feature 227,3
26380,5233,feature 228,35
26372,5233,feature 229,28
26374,5233,feature 230,20
26384,5233,feature 231,1
26377,5233,feature 232,68
26387,5233,feature 233,15
26373,5233,feature 234,28
26383,5233,feature 235,12
26370,5233,feature 236,3


In [ ]:
log_data_enc_grp.index.value_counts().sort_values(ascending=False)

1        1
145      1
4        1
5        1
6        1
        ..
18550    1
18551    1
18545    1
18518    1
18552    1
Name: id, Length: 18552, dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler

volumn_scalar = StandardScaler()
log_data['volumn_enc']= volumn_scalar.fit_transform(np.array(log_data.volume).reshape(-1, 1))
log_data

,id,log_feature,volume,volumn_enc
0,6597,feature 68,6,-0.134922
1,8011,feature 68,7,-0.098311
2,2597,feature 68,1,-0.317977
3,5022,feature 172,2,-0.281366
4,5022,feature 56,1,-0.317977
...,...,...,...,...
58666,8720,feature 209,1,-0.317977
58667,6488,feature 54,3,-0.244755
58668,878,feature 62,1,-0.317977
58669,4464,feature 209,1,-0.317977


#### resource_type_file
- Contains ID and Resource Type
- Repeation of ID indicates multile resource type are assiciated with one ID. 
- Approach
  - Binarize the resource type 
  - group by ID  and sum operation
- Result
  - Remove duplication of ID. one row per id 
  - rows are sparse with 1 at resource type position associated with ID

In [ ]:
resource_data = pd.read_csv(resource_type_file)
resource_data.head()

,id,resource_type
0,6597,resource_type 8
1,8011,resource_type 8
2,2597,resource_type 8
3,5022,resource_type 8
4,6852,resource_type 8


In [ ]:
resource_data.shape

(21076, 2)

In [ ]:
resource_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21076 entries, 0 to 21075
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             21076 non-null  int64 
 1   resource_type  21076 non-null  object
dtypes: int64(1), object(1)
memory usage: 329.4+ KB


In [ ]:
resource_data.id.value_counts().sort_values(ascending=False)

11595    5
7194     5
1379     5
7378     5
11055    5
        ..
2698     1
7254     1
4923     1
11216    1
4464     1
Name: id, Length: 18552, dtype: int64

In [ ]:
resource_data_enc = pd.get_dummies(resource_data).groupby(by='id').sum()
resource_data_enc

,resource_type_resource_type 1,resource_type_resource_type 10,resource_type_resource_type 2,resource_type_resource_type 3,resource_type_resource_type 4,resource_type_resource_type 5,resource_type_resource_type 6,resource_type_resource_type 7,resource_type_resource_type 8,resource_type_resource_type 9
id,,,,,,,,,,
1,0,0,0,0,0,0,1,0,1,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
18548,0,0,1,0,0,0,0,0,0,0
18549,0,0,1,0,0,0,0,0,0,0
18550,0,0,0,0,0,0,0,0,1,0


#### severity_type_file
- Contains ID ans severity type
- No Repetation of ID
- Approach
  - Binarize the severity type 

In [ ]:
severity_data = pd.read_csv(severity_type_file)
severity_data

,id,severity_type
0,6597,severity_type 2
1,8011,severity_type 2
2,2597,severity_type 2
3,5022,severity_type 1
4,6852,severity_type 1
...,...,...
18547,3761,severity_type 1
18548,8720,severity_type 1
18549,6488,severity_type 2
18550,878,severity_type 2


In [ ]:
severity_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18552 entries, 0 to 18551
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             18552 non-null  int64 
 1   severity_type  18552 non-null  object
dtypes: int64(1), object(1)
memory usage: 290.0+ KB


In [ ]:
severity_data.id.value_counts().sort_values(ascending=False)

6597     1
6208     1
5022     1
6852     1
5611     1
        ..
6488     1
878      1
15084    1
2557     1
4464     1
Name: id, Length: 18552, dtype: int64

In [ ]:
severity_data.severity_type.value_counts().sort_values(ascending=False)

severity_type 2    8737
severity_type 1    8728
severity_type 4    1014
severity_type 5      65
severity_type 3       8
Name: severity_type, dtype: int64

In [ ]:
severity_data_enc = pd.get_dummies(severity_data)
severity_data_enc

,id,severity_type_severity_type 1,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5
0,6597,0,1,0,0,0
1,8011,0,1,0,0,0
2,2597,0,1,0,0,0
3,5022,1,0,0,0,0
4,6852,1,0,0,0,0
...,...,...,...,...,...,...
18547,3761,1,0,0,0,0
18548,8720,1,0,0,0,0
18549,6488,0,1,0,0,0
18550,878,0,1,0,0,0


#### Merging all the data together
- Merge all Data together using left join on id column

In [ ]:
train_data_1 = train_data.merge(severity_data_enc, how='left', on='id')
train_data_1

,id,location,fault_severity,location_enc,severity_type_severity_type 1,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5
0,14121,location 118,1,129,0,1,0,0,0
1,9320,location 91,0,812,0,1,0,0,0
2,14394,location 152,1,161,0,1,0,0,0
3,8218,location 931,1,830,1,0,0,0,0
4,14804,location 120,0,132,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
5994,15060,location 803,1,712,1,0,0,0,0
5995,4587,location 879,0,781,1,0,0,0,0
5996,6846,location 976,1,869,0,1,0,0,0
5997,14207,location 1053,1,51,1,0,0,0,0


In [ ]:
train_data_2 = train_data_1.merge(resource_data_enc, how='left', on='id')
train_data_2

,id,location,fault_severity,location_enc,severity_type_severity_type 1,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5,resource_type_resource_type 1,resource_type_resource_type 10,resource_type_resource_type 2,resource_type_resource_type 3,resource_type_resource_type 4,resource_type_resource_type 5,resource_type_resource_type 6,resource_type_resource_type 7,resource_type_resource_type 8,resource_type_resource_type 9
0,14121,location 118,1,129,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1,9320,location 91,0,812,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
2,14394,location 152,1,161,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
3,8218,location 931,1,830,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,14804,location 120,0,132,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,15060,location 803,1,712,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5995,4587,location 879,0,781,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5996,6846,location 976,1,869,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
5997,14207,location 1053,1,51,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
train_data_3 = train_data_2.merge(event_data_enc, how='inner', on='id')
train_data_3

,id,location,fault_severity,location_enc,severity_type_severity_type 1,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5,resource_type_resource_type 1,...,event_type_event_type 5,event_type_event_type 50,event_type_event_type 51,event_type_event_type 52,event_type_event_type 53,event_type_event_type 54,event_type_event_type 6,event_type_event_type 7,event_type_event_type 8,event_type_event_type 9
0,14121,location 118,1,129,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9320,location 91,0,812,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14394,location 152,1,161,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8218,location 931,1,830,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14804,location 120,0,132,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,15060,location 803,1,712,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5995,4587,location 879,0,781,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5996,6846,location 976,1,869,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5997,14207,location 1053,1,51,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data_4 = train_data_3.merge(log_data_enc_grp, how='left', on='id')
train_data_4

,id,location,fault_severity,location_enc,severity_type_severity_type 1,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5,resource_type_resource_type 1,...,377,378,379,380,381,382,383,384,385,386
0,14121,location 118,1,129,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9320,location 91,0,812,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14394,location 152,1,161,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8218,location 931,1,830,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14804,location 120,0,132,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,15060,location 803,1,712,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5995,4587,location 879,0,781,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5996,6846,location 976,1,869,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5997,14207,location 1053,1,51,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data_4.columns.values

array(['id', 'location', 'fault_severity', 'location_enc',
       'severity_type_severity_type 1', 'severity_type_severity_type 2',
       'severity_type_severity_type 3', 'severity_type_severity_type 4',
       'severity_type_severity_type 5', 'resource_type_resource_type 1',
       'resource_type_resource_type 10', 'resource_type_resource_type 2',
       'resource_type_resource_type 3', 'resource_type_resource_type 4',
       'resource_type_resource_type 5', 'resource_type_resource_type 6',
       'resource_type_resource_type 7', 'resource_type_resource_type 8',
       'resource_type_resource_type 9', 'event_type_event_type 1',
       'event_type_event_type 10', 'event_type_event_type 11',
       'event_type_event_type 12', 'event_type_event_type 13',
       'event_type_event_type 14', 'event_type_event_type 15',
       'event_type_event_type 17', 'event_type_event_type 18',
       'event_type_event_type 19', 'event_type_event_type 2',
       'event_type_event_type 20', 'event_type_e

### Create X_train, Y_train, X_test, y_test

In [ ]:
pred_col = 'fault_severity'
col_to_drop = ['id', 'location', 'fault_severity', 'severity_type_severity_type 1', 'resource_type_resource_type 1', 'event_type_event_type 1', 1, 'location_enc']
random_state=21

In [ ]:
y = train_data_4[pred_col]
X = train_data_4.drop(columns=col_to_drop)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=random_state)
print(f'X_train shape:{X_train.shape}')
print(f'y_train shape:{y_train.shape}')
print(f'X_test shape:{X_test.shape}')
print(f'y_test shape:{y_test.shape}')

X_train shape:(4799, 450)
y_train shape:(4799,)
X_test shape:(1200, 450)
y_test shape:(1200,)


In [ ]:
X_train.head()

,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5,resource_type_resource_type 10,resource_type_resource_type 2,resource_type_resource_type 3,resource_type_resource_type 4,resource_type_resource_type 5,resource_type_resource_type 6,...,377,378,379,380,381,382,383,384,385,386
743,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3369,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_train.columns.values

array(['severity_type_severity_type 2', 'severity_type_severity_type 3',
       'severity_type_severity_type 4', 'severity_type_severity_type 5',
       'resource_type_resource_type 10', 'resource_type_resource_type 2',
       'resource_type_resource_type 3', 'resource_type_resource_type 4',
       'resource_type_resource_type 5', 'resource_type_resource_type 6',
       'resource_type_resource_type 7', 'resource_type_resource_type 8',
       'resource_type_resource_type 9', 'event_type_event_type 10',
       'event_type_event_type 11', 'event_type_event_type 12',
       'event_type_event_type 13', 'event_type_event_type 14',
       'event_type_event_type 15', 'event_type_event_type 17',
       'event_type_event_type 18', 'event_type_event_type 19',
       'event_type_event_type 2', 'event_type_event_type 20',
       'event_type_event_type 21', 'event_type_event_type 22',
       'event_type_event_type 23', 'event_type_event_type 24',
       'event_type_event_type 25', 'event_type_event_

### Model Building and Training

In [ ]:
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(multi_class='ovr', solver='lbfgs', class_weight='balanced', verbose= 1, random_state=random_state, max_iter=10000)
log_clf.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    9.1s finished


LogisticRegression(class_weight='balanced', max_iter=10000, multi_class='ovr',
                   random_state=21, verbose=1)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

print(f'Train_Accuracy : {accuracy_score(y_train, log_clf.predict(X_train))}')
print(f'Test_Accuracy : {accuracy_score(y_test, log_clf.predict(X_test))}')

Train_Accuracy : 0.682225463638258
Test_Accuracy : 0.6525


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
print(f'Train ROC AUC Score : {roc_auc_score(y_train, log_clf.predict_proba(X_train), multi_class="ovr")}')
print(f'Test ROC AUC Score : {roc_auc_score(y_test, log_clf.predict_proba(X_test), multi_class="ovr")}')

Train ROC AUC Score : 0.8598868174485049
Test ROC AUC Score : 0.8217860662042957


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
print(f'Train confusion Matrix: \n { confusion_matrix(y_train, log_clf.predict(X_train) )}')
print(f'Test confusion Matrix: \n { confusion_matrix(y_test, log_clf.predict(X_test) )}')

Train confusion Matrix: 
 [[2008  890  214]
 [ 201  840  150]
 [   4   66  426]]
Test confusion Matrix: 
 [[503 221  55]
 [ 63 206  50]
 [  8  20  74]]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
log_clf.predict_proba(X_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


array([[6.98495241e-01, 3.01473061e-01, 3.16980248e-05],
       [8.64575238e-01, 1.35318926e-01, 1.05835919e-04],
       [2.78279464e-01, 4.32775564e-01, 2.88944972e-01],
       ...,
       [2.92384180e-01, 2.95660188e-01, 4.11955632e-01],
       [7.77733485e-02, 7.67905988e-01, 1.54320664e-01],
       [7.74983025e-01, 1.82930355e-01, 4.20866200e-02]])

### test_file prediction - submission

In [ ]:
test_data = pd.read_csv(test_file)
test_data

,id,location
0,4751,location 242
1,7527,location 714
2,2028,location 645
3,14721,location 155
4,14727,location 91
...,...,...
1377,870,location 167
1378,18068,location 106
1379,14111,location 1086
1380,15189,location 7


In [ ]:
# merge the file with other data
test_data_1 = test_data.merge(severity_data_enc, how='left', on='id').merge(resource_data_enc, how='left', on='id').merge(event_data_enc, how='inner', on='id').merge(log_data_enc_grp, how='left', on='id')
test_data_1

,id,location,severity_type_severity_type 1,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5,resource_type_resource_type 1,resource_type_resource_type 10,resource_type_resource_type 2,...,377,378,379,380,381,382,383,384,385,386
0,4751,location 242,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,7527,location 714,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2028,location 645,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,14721,location 155,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,14727,location 91,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,870,location 167,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1378,18068,location 106,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1379,14111,location 1086,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1380,15189,location 7,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_col_to_drop = ['id', 'location', 'severity_type_severity_type 1', 'resource_type_resource_type 1', 'event_type_event_type 1', 1]

test_data_2 = test_data_1.drop(columns=test_col_to_drop)
test_data_2

,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5,resource_type_resource_type 10,resource_type_resource_type 2,resource_type_resource_type 3,resource_type_resource_type 4,resource_type_resource_type 5,resource_type_resource_type 6,...,377,378,379,380,381,382,383,384,385,386
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1378,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1379,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_prob = log_clf.predict_proba(test_data_2)
test_prob

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


array([[1.00000000e+00, 2.75256626e-17, 3.98377119e-23],
       [1.44112913e-01, 5.04009337e-01, 3.51877750e-01],
       [2.30997491e-01, 6.98629183e-01, 7.03733263e-02],
       ...,
       [1.45688136e-01, 3.16670039e-01, 5.37641825e-01],
       [5.04548414e-01, 9.92605184e-02, 3.96191068e-01],
       [4.64663301e-01, 4.72363375e-01, 6.29733243e-02]])

In [ ]:
final_op = np.hstack([np.array(test_data_1.id.values).reshape(-1, 1), test_prob])
final_op

array([[4.75100000e+03, 1.00000000e+00, 2.75256626e-17, 3.98377119e-23],
       [7.52700000e+03, 1.44112913e-01, 5.04009337e-01, 3.51877750e-01],
       [2.02800000e+03, 2.30997491e-01, 6.98629183e-01, 7.03733263e-02],
       ...,
       [1.41110000e+04, 1.45688136e-01, 3.16670039e-01, 5.37641825e-01],
       [1.51890000e+04, 5.04548414e-01, 9.92605184e-02, 3.96191068e-01],
       [1.70670000e+04, 4.64663301e-01, 4.72363375e-01, 6.29733243e-02]])

In [ ]:
final_op

array([[4.75100000e+03, 1.00000000e+00, 2.75256626e-17, 3.98377119e-23],
       [7.52700000e+03, 1.44112913e-01, 5.04009337e-01, 3.51877750e-01],
       [2.02800000e+03, 2.30997491e-01, 6.98629183e-01, 7.03733263e-02],
       ...,
       [1.41110000e+04, 1.45688136e-01, 3.16670039e-01, 5.37641825e-01],
       [1.51890000e+04, 5.04548414e-01, 9.92605184e-02, 3.96191068e-01],
       [1.70670000e+04, 4.64663301e-01, 4.72363375e-01, 6.29733243e-02]])

In [ ]:
final_op_df= pd.DataFrame(final_op)
final_op_df.rename(columns={
    0:'id',
    1:'predict_0',
    2:'predict_1',
    3:'predict_2'
}, inplace=True)
final_op_df['id'] = final_op_df.id.astype('int32')
final_op_df


,id,predict_0,predict_1,predict_2
0,4751,1.000000,2.752566e-17,3.983771e-23
1,7527,0.144113,5.040093e-01,3.518777e-01
2,2028,0.230997,6.986292e-01,7.037333e-02
3,14721,0.761786,2.381861e-01,2.800822e-05
4,14727,1.000000,2.865260e-46,3.542360e-13
...,...,...,...,...
1377,870,0.463715,5.357732e-01,5.116675e-04
1378,18068,0.558627,4.027931e-01,3.857957e-02
1379,14111,0.145688,3.166700e-01,5.376418e-01
1380,15189,0.504548,9.926052e-02,3.961911e-01


In [ ]:
final_op_df.to_csv(
    os.path.join(base_folder, 'submission.csv'),
    index=False
)

Thanks For Reading. Please leave your suggestion if any